In [1]:
!pip install opencv-python

  Using cached opencv_python-4.5.1.48-cp37-cp37m-win_amd64.whl (34.9 MB)


In [2]:
import cv2
import os
import numpy as np

In [3]:
def getOutputsNames(net):
    layersNames = net.getLayerNames()
    return [layersNames[i[0] -1] for i in net.getUnconnectedOutLayers()]

In [4]:
def postprocess(img, outs, classIds, boxes):
    confidences = []
    for i in range(len(outs)):
        data = outs[i]
        for j in range(len(data)):
            scores = outs[i][j][5:]
            _, confidence, _, classIdPoint = cv2.minMaxLoc(scores)
            if (confidence > CONFTHRESHOLD):
                centerX = int(data[j][0] * len(img[0]))
                centerY = int(data[j][1] * len(img))
                width = int(data[j][2] * len(img[0]))
                height = int(data[j][3] * len(img))
                left = int(centerX - width/2)
                top = int(centerY - height/2)
                
                classIds.append(classIdPoint[1])
                confidences.append(confidence)
                boxes.append([left, top, width, height])
    indices = cv2.dnn.NMSBoxes(boxes, confidences, CONFTHRESHOLD, NMSTHRESHOLD)
    return indices
            

In [5]:
CONFTHRESHOLD = 0.5
NMSTHRESHOLD = 0.4
INPWIDTH = 416
INPHEIGHT = 416
classes = []
f = open("obj.txt", "r")
while (1):
    line = f.readline()
    if not line:
        break;
    classes.append(line[:-1])
print(classes)
    
directory = "original"
IMGLIST = []
for root,dirs,files in os.walk(directory):
    for file in files:
        if(file[-3:] != 'txt'):
            IMGLIST.append(os.path.join(root,file))

['Apple', 'Peach', 'Pomegranate', 'Lemon', 'Mango', 'Orange', 'pear', 'Strawberry', 'Tomato', 'Watermelon']


In [6]:
for i in range(len(classes)):
    try: 
        os.mkdir(os.path.join("crop",classes[i])) 
    except OSError as error: 
        print(error)   

[WinError 183] Cannot create a file when that file already exists: 'crop\\Apple'
[WinError 183] Cannot create a file when that file already exists: 'crop\\Peach'
[WinError 183] Cannot create a file when that file already exists: 'crop\\Pomegranate'
[WinError 183] Cannot create a file when that file already exists: 'crop\\Lemon'
[WinError 183] Cannot create a file when that file already exists: 'crop\\Mango'
[WinError 183] Cannot create a file when that file already exists: 'crop\\Orange'
[WinError 183] Cannot create a file when that file already exists: 'crop\\pear'
[WinError 183] Cannot create a file when that file already exists: 'crop\\Strawberry'
[WinError 183] Cannot create a file when that file already exists: 'crop\\Tomato'
[WinError 183] Cannot create a file when that file already exists: 'crop\\Watermelon'


In [7]:
net = cv2.dnn.readNet("yolo-fastest_last.weights", "yolo-fastest.cfg")
net.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA)
net.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA_FP16)

In [8]:
x = 0
for i in range(len(IMGLIST)):
    img = cv2.imread(IMGLIST[i])
    blob = cv2.dnn.blobFromImage(img, 1/255.0, (INPWIDTH, INPHEIGHT), (0,0,0), True, False)
    net.setInput(blob)

    outs = net.forward(getOutputsNames(net))

    boxes = []
    classIds = []
    indices = postprocess(img, outs, classIds, boxes)
    print("wtf", indices)
    for j in range(len(indices)):
        x += 1
        top = max(boxes[np.squeeze(indices[j])][1],0)
        bottom = min(boxes[np.squeeze(indices[j])][1]+boxes[np.squeeze(indices[j])][3], len(img))
        left = max(boxes[np.squeeze(indices[j])][0],0)
        right = min(boxes[np.squeeze(indices[j])][0]+boxes[np.squeeze(indices[j])][2], len(img[0]))
        crop = img[top:bottom, left:right]
        cv2.imwrite("crop/"+ classes[classIds[np.squeeze(indices[j])]] + "/"+str(x)+".jpg", crop)

In [9]:
while cv2.waitKey(10) < 1:
    (grabbed, frame) = cv.read()
    if not grabbed:
        exit()

    start = time.time()
    classes, scores, boxes = model.detect(frame, CONFIDENCE_THRESHOLD, NMS_THRESHOLD)
    end = time.time()

    start_drawing = time.time()
    for (classid, score, box) in zip(classes, scores, boxes):
        color = COLORS[int(classid) % len(COLORS)]
        label = "%s : %f" % (class_names[classid[0]], score)
        cv2.rectangle(frame, box, color, 2)
        cv2.putText(frame, label, (box[0], box[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
    end_drawing = time.time()
    
    fps_label = "FPS: %.2f (excluding drawing time of %.2fms)" % (1 / (end - start), (end_drawing - start_drawing) * 1000)
    cv2.putText(frame, fps_label, (0, 25), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2)
    cv2.imshow("detections", frame)

NameError: name 'cv' is not defined